## Notebook Magic

In [120]:
%matplotlib inline
%load_ext autoreload

## Imports

In [121]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

from sklearn.cluster import KMeans

## Load Intermediate Layer

In [122]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/intermediate.parquet")
df.head()

,user_id,journey_count,register_date,register_reason,exit_date_21,exit_date_31,exit_reason,birth_date,gender,age,...,intervention_date_791,intervention_date_794,intervention_date_795,intervention_date_796,intervention_date_797,intervention_date_9902,intervention_date_9903,intervention_date_9904,intervention_date_9906,intervention_date_9907
0,186,1.0,2018-05-03,5.0,NaT,2018-07-02,19.0,1966-04-28,F,52.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,304,1.0,2017-01-17,7.0,NaT,2017-11-21,11.0,1962-06-28,M,54.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,323,1.0,2016-02-05,7.0,NaT,2017-07-25,20.0,1979-01-22,M,37.0,...,NaT,NaT,NaT,NaT,NaT,2016-09-23,NaT,NaT,NaT,NaT
3,574,1.0,2017-06-30,5.0,NaT,2018-05-18,61.0,1976-05-06,M,41.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,724,3.0,2018-04-18,15.0,NaT,2018-12-19,44.0,1978-12-13,F,39.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


## Get intervention codes 

In [123]:
interventions_cols = [col for col in df.columns if "intervention" in col]

In [124]:
target_codes = [codes.replace('intervention_date_', '') for codes in interventions_cols]

In [125]:
# Load data dictionary
column_dict = yaml.load(open('../conf/base/intervention_translation.yml'),Loader=yaml.FullLoader)

FileNotFoundError: [Errno 2] No such file or directory: '../conf/base/intervention_translation.yml'

In [ ]:
print(len(set(column_dict)))
print(len(set(target_codes)))

In [ ]:
target_codes = list(map(int, target_codes))

## Only keep dict keys where key in target code

In [ ]:
unwanted = set(column_dict) - set(target_codes)
for unwanted_key in unwanted: del column_dict[unwanted_key]

In [ ]:
len(column_dict)

In [ ]:
# write dict:
with open('intervention_translation_dict.yml', 'w') as outfile:
    yaml.dump(column_dict, outfile, default_flow_style=False)

## Use new dict to translate intermediate column names

In [ ]:
# Load new data dictionary
code_dict = yaml.load(open('../conf/base/intervention_translation_dict.yml'),Loader=yaml.FullLoader)
df = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/intermediate.parquet")

In [ ]:
df.head()

In [ ]:
# replace for intervention cols (as in same order)

In [ ]:
df.columns = [col.replace("intervention_date_", "") for col in df.columns]

In [ ]:
code_dict = {str(key):code_dict[key] for key in code_dict}

In [ ]:
# replace col_names
df = df.rename(columns=code_dict)

In [ ]:
def Translate_Intervention_Codes(df):
    '''Function to '''
    code_dict = yaml.load(open('../conf/base/intervention_translation_dict.yml'),Loader=yaml.FullLoader)
    df.columns = [col.replace("intervention_date_", "") for col in df.columns]
    code_dict = {str(key):code_dict[key] for key in code_dict}
    df = df.rename(columns=code_dict)
    return df

In [ ]:
df = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/intermediate.parquet")

In [ ]:
df = translate_codes(df)
    

In [127]:
m = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/modelling.parquet")
m.columns.values

array(['user_id', 'journey_count', 'register_date', 'register_reason',
       'exit_date_21', 'exit_date_31', 'exit_reason', 'birth_date',
       'gender', 'age', 'disabled', 'dependents', 'professional_training',
       'profession_card', 'school_qualification', 'college_qualification',
       'civil_status', 'registration_type', 'nationality', 'parish',
       'center_area', 'subsidy', 'rsi', 'previous_cpp',
       'previous_job_experience', 'desired_cpp', 'desired_location',
       'desired_work_time', 'desired_contract', 'success', 'ttj_sub_9',
       'rvcc_other_entities', 'job_search_techn', 'professional_rvcc',
       'rvcc_integrated', 'train_active_life',
       'tutoring_in_individual_job_search', 'job_search_assistance',
       'pae_cpe_subsidy', 'paper_subsidy_ace', 'paecpe_ace',
       'business_and_employment_creation_help',
       'invest_young_target_promoter', 'invest_young_another_promoter',
       'other_employment_program', 'insertion_companies',
       'professiona